In [39]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [40]:
df1= pd.read_csv('data.csv')
df1.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [41]:
df2= df1.drop(['country','view','sqft_above','sqft_basement','date','waterfront','street','statezip'],axis='columns')
df2.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,yr_built,yr_renovated,city
0,313000.0,3.0,1.50,1340,7912,1.5,3,1955,2005,Shoreline
1,2384000.0,5.0,2.50,3650,9050,2.0,5,1921,0,Seattle
2,342000.0,3.0,2.00,1930,11947,1.0,4,1966,0,Kent
3,420000.0,3.0,2.25,2000,8030,1.0,4,1963,0,Bellevue
4,550000.0,4.0,2.50,1940,10500,1.0,4,1976,1992,Redmond


In [42]:
def make_int(x):
    return int(x)

In [43]:
df3= df2.copy()
df3['bathrooms']= df3['bathrooms'].apply(make_int)
df3['bedrooms']= df3['bedrooms'].apply(make_int)
df3['floors']= df3['floors'].apply(make_int)
df3.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,yr_built,yr_renovated,city
0,313000.0,3,1,1340,7912,1,3,1955,2005,Shoreline
1,2384000.0,5,2,3650,9050,2,5,1921,0,Seattle
2,342000.0,3,2,1930,11947,1,4,1966,0,Kent
3,420000.0,3,2,2000,8030,1,4,1963,0,Bellevue
4,550000.0,4,2,1940,10500,1,4,1976,1992,Redmond


In [44]:
df4= df3.copy()
df4['price_per_sqft']= df4['price']/df4['sqft_living']
df4.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,yr_built,yr_renovated,city,price_per_sqft
0,313000.0,3,1,1340,7912,1,3,1955,2005,Shoreline,233.582090
1,2384000.0,5,2,3650,9050,2,5,1921,0,Seattle,653.150685
2,342000.0,3,2,1930,11947,1,4,1966,0,Kent,177.202073
3,420000.0,3,2,2000,8030,1,4,1963,0,Bellevue,210.000000
4,550000.0,4,2,1940,10500,1,4,1976,1992,Redmond,283.505155


In [45]:
df5= df4[~(df3.sqft_living/df3.bedrooms<300)]
df5.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,yr_built,yr_renovated,city,price_per_sqft
0,313000.0,3,1,1340,7912,1,3,1955,2005,Shoreline,233.582090
1,2384000.0,5,2,3650,9050,2,5,1921,0,Seattle,653.150685
2,342000.0,3,2,1930,11947,1,4,1966,0,Kent,177.202073
3,420000.0,3,2,2000,8030,1,4,1963,0,Bellevue,210.000000
4,550000.0,4,2,1940,10500,1,4,1976,1992,Redmond,283.505155


In [46]:
df5.price_per_sqft.describe()

count     4565.000000
mean       265.259366
std        357.823300
min          0.000000
25%        180.555556
50%        243.478261
75%        314.634146
max      22533.898305
Name: price_per_sqft, dtype: float64

In [47]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('city'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df6 = remove_pps_outliers(df5)
df6.shape

(3812, 11)

In [48]:
df7= df6.drop(['price_per_sqft'],axis='columns')
df7.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,yr_built,yr_renovated,city
0,100000.0,2,1,910,22000,1,3,1956,2001,Algona
1,262000.0,4,2,2020,6236,2,3,2002,0,Algona
2,248000.0,4,3,2163,5883,2,3,2006,0,Algona
3,196440.0,3,2,1560,7352,1,3,1992,0,Algona
4,367500.0,4,3,3110,7231,2,3,1997,0,Auburn


In [49]:
dummies = pd.get_dummies(df7.city)
df8 = pd.concat([df7,dummies],axis='columns')
df9 = df8.drop('city',axis='columns')
df9.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,yr_built,yr_renovated,Algona,...,Sammamish,SeaTac,Seattle,Shoreline,Skykomish,Snoqualmie,Tukwila,Vashon,Woodinville,Yarrow Point
0,100000.0,2,1,910,22000,1,3,1956,2001,1,...,0,0,0,0,0,0,0,0,0,0
1,262000.0,4,2,2020,6236,2,3,2002,0,1,...,0,0,0,0,0,0,0,0,0,0
2,248000.0,4,3,2163,5883,2,3,2006,0,1,...,0,0,0,0,0,0,0,0,0,0
3,196440.0,3,2,1560,7352,1,3,1992,0,1,...,0,0,0,0,0,0,0,0,0,0
4,367500.0,4,3,3110,7231,2,3,1997,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
X = df9.drop(['price'],axis='columns')
X.head(3)

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,yr_built,yr_renovated,Algona,Auburn,...,Sammamish,SeaTac,Seattle,Shoreline,Skykomish,Snoqualmie,Tukwila,Vashon,Woodinville,Yarrow Point
0,2,1,910,22000,1,3,1956,2001,1,0,...,0,0,0,0,0,0,0,0,0,0
1,4,2,2020,6236,2,3,2002,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,4,3,2163,5883,2,3,2006,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
y = df9.price
y.head(3)

0    100000.0
1    262000.0
2    248000.0
Name: price, dtype: float64

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [58]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

0.7764824012606717

In [59]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

array([0.8064661 , 0.82619385, 0.82053456, 0.79039411, 0.8262727 ])

In [73]:
def predict_price(bed,bath,sqft_liv,sqft_lot,floor,cond,yr_built,yr_reno,city):    
    city_index = np.where(X.columns==city)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = bed
    x[1] = bath
    x[2] = sqft_liv
    x[3] = sqft_lot
    x[4] = floor
    x[5] = cond
    x[6] = yr_built
    x[7] = yr_reno
    if city_index >= 0:
        x[city_index] = 1

    return lr_clf.predict([x])[0]

In [74]:
predict_price(4,4,2165,5890,2,4,2005,0,'Seattle')

650723.3822951315

In [76]:
import pickle
with open('Home_prices_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)